# Make a smaller data set for testing models

In [1]:
library(nakedpipe)
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.1
✔ tidyr   1.1.1     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [20]:
data_dir <- file.path("..", "modeling_data")
modeling_data_path <- file.path(data_dir, "depmap_modeling_dataframe.csv")
out_path <- file.path(data_dir, "depmap_modeling_dataframe_subsample.csv")

In [4]:
modeling_data <- read_csv(modeling_data_path, guess_max = 1e5)
head(modeling_data)

Parsed with column specification:
cols(
  .default = col_character(),
  lfc = col_double(),
  pdna_batch = col_double(),
  passes_qc = col_logical(),
  n_alignments = col_double(),
  chrom_pos = col_double(),
  segment_mean = col_double(),
  copy_number = col_double(),
  n_muts = col_double(),
  any_deleterious = col_logical(),
  mutated_at_guide_location = col_logical(),
  rna_expr = col_double()
)

See spec(...) for full column specifications.



sgrna,replicate_id,lfc,pdna_batch,passes_qc,depmap_id,primary_or_metastasis,lineage,lineage_subtype,kras_mutation,⋯,segment_mean,copy_number,n_muts,any_deleterious,variant_classification,is_deleterious,is_tcga_hotspot,is_cosmic_hotspot,mutated_at_guide_location,rna_expr
<chr>,<chr>,<dbl>,<dbl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<dbl>
AAAAAAATCCAGCAATGCAG,143b-311cas9_repa_p6_batch3,0.2896938,3,TRUE,ACH-001001,Primary,bone,osteosarcoma,G12S,⋯,1.1434280,2.209053,0,FALSE,NA,NA,NA,NA,FALSE,4.1009776
AAAAAACCCGTAGATAGCCT,143b-311cas9_repa_p6_batch3,0.1701723,3,TRUE,ACH-001001,Primary,bone,osteosarcoma,G12S,⋯,0.8850587,1.846840,0,FALSE,NA,NA,NA,NA,FALSE,7.4709435
AAAAAAGAAGAAAAAACCAG,143b-311cas9_repa_p6_batch3,-0.6959467,3,TRUE,ACH-001001,Primary,bone,osteosarcoma,G12S,⋯,0.8946238,1.859125,0,FALSE,NA,NA,NA,NA,FALSE,4.6270231
AAAAAAGCTCAAGAAGGAGG,143b-311cas9_repa_p6_batch3,-0.3249354,3,TRUE,ACH-001001,Primary,bone,osteosarcoma,G12S,⋯,1.0461620,2.065029,0,FALSE,NA,NA,NA,NA,FALSE,4.7750505
AAAAAAGGCTGTAAAAGCGT,143b-311cas9_repa_p6_batch3,0.1428739,3,TRUE,ACH-001001,Primary,bone,osteosarcoma,G12S,⋯,0.8744568,1.833318,0,FALSE,NA,NA,NA,NA,FALSE,0.2265085
AAAAAAGGGCTCCAAAAAGG,143b-311cas9_repa_p6_batch3,-0.2998787,3,TRUE,ACH-001001,Primary,bone,osteosarcoma,G12S,⋯,1.0855960,2.122252,0,FALSE,NA,NA,NA,NA,FALSE,0.3785116


In [5]:
glimpse(modeling_data)

Rows: 120,235,860
Columns: 25
$ sgrna                     <chr> "AAAAAAATCCAGCAATGCAG", "AAAAAACCCGTAGATAGC…
$ replicate_id              <chr> "143b-311cas9_repa_p6_batch3", "143b-311cas…
$ lfc                       <dbl> 0.28969381, 0.17017231, -0.69594665, -0.324…
$ pdna_batch                <dbl> 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3…
$ passes_qc                 <lgl> TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, T…
$ depmap_id                 <chr> "ACH-001001", "ACH-001001", "ACH-001001", "…
$ primary_or_metastasis     <chr> "Primary", "Primary", "Primary", "Primary",…
$ lineage                   <chr> "bone", "bone", "bone", "bone", "bone", "bo…
$ lineage_subtype           <chr> "osteosarcoma", "osteosarcoma", "osteosarco…
$ kras_mutation             <chr> "G12S", "G12S", "G12S", "G12S", "G12S", "G1…
$ genome_alignment          <chr> "chr10_110964620_+", "chr12_95003615_+", "c…
$ n_alignments              <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ hugo_symbol         

In [6]:
# Number of unique guides.
n_distinct(modeling_data$sgrna)

[1] 72133

In [7]:
# Number of unique cell lines.
n_distinct(modeling_data$depmap_id)

[1] 767

In [8]:
# Number of genes.
n_distinct(modeling_data$hugo_symbol)

[1] 18524

In [12]:
# Some specific genes to subset.
specific_genes <- c("KRAS", "BRAF", "PIK3CA", "PTK2", "MDM2", "TP53")

set.seed(0)
num_random_genes <- 20
subsample_genes <- c(
    sample(unique(modeling_data$hugo_symbol), num_random_genes),
    specific_genes
)

# Check that all genes are actually in the modeling data.
all(subsample_genes %in% modeling_data$hugo_symbol)

[1] TRUE

In [13]:
subsample_genes

[1] "PHACTR3"   "ZSWIM8"    "DPH7"      "LGALS7B"   "DISP1"     "KIF3C"    
 [7] "RNF125"    "LGALS4"    "SLC7A14"   "UQCRC1"    "SCMH1"     "SMAD7"    
[13] "GHSR"      "IQCK"      "NDUFAF3"   "FAM43B"    "PDE5A"     "HIST1H2BO"
[19] "ADAMTS13"  "CXCL2"     "KRAS"      "BRAF"      "PIK3CA"    "PTK2"     
[25] "MDM2"      "TP53"

In [18]:
subsample_modeling_data <- modeling_data %>% filter(hugo_symbol %in% subsample_genes)
pryr::object_size(subsample_modeling_data)

31.5 MB

In [21]:
nrow(subsample_modeling_data)

[1] 166345

In [22]:
write_csv(subsample_modeling_data, out_path)